#QCHack 2022 IBM Challenge

### Authors: Ruihao, Guarav, Muralik, Kevin, Adrian


In this project we create a new quantum implementation of Snakes and Ladders to compete with a classical player and demonstrate a quantum advantage.  We use the Qiskit programming language to create a board represented by a basis of 4 qubits that the quantum player can travel through in a superposition of paths. We then utilize a generalization of Grover's Algorithm to amplify the amplitudes of the leading paths.

Though we start with a 16 square board with two ladders and two snakes, this can be generalized to arbitrary boards with varying levels of quantum advantage over a classical player, representing different levels of difficulty.

In [ ]:
!pip install qiskit==0.35

In [ ]:
!pip install pylatexenc

In [ ]:
import numpy as np
import qiskit
from qiskit import *
import random

In [72]:
"""
Create a predetermined 4-by-4 board with snakes and ladders at several coordinates.
Use random number generator to simulate a coin toss

position of ladders: [1] and [4]. The ladder operation should follow: 1 --> 5, 4 --> 13
position of snakes: [10], [14]. The snake operation should follow: 10 --> 3, 14 --> 0



"""

class quantum_snake_ladder:
  def __init__(self, grid_size=4, total_coin_toss = 5, ladder_start = [1,4], ladder_end=[5,13], snake_head=[14,10], snake_tail=[0,3]):
    self.grid_size = grid_size
    self.total_coin_toss = total_coin_toss
    self.ladder_start = ladder_start
    self.ladder_end = ladder_end
    self.snake_head = snake_head
    self.snake_tail = snake_tail
    if len(self.ladder_start) != len(self.ladder_end):
      raise ValueError("total ladder start points not equal to ladder end points")
    if len(self.snake_head) != len(self.snake_tail):
      raise ValueError("total snake heads not equal to snake tails")
  
    self.Total_elem = self.grid_size**2
    self.Max_bits = len(bin(self.Total_elem).replace('0b',''))

    self.current_position = [0]
    self.current_classical_position = [0]

    qr_main = QuantumRegister(self.Max_bits, name='q')
    anc_main = QuantumRegister(1, name='anc')
    circ_main = QuantumCircuit(qr_main, anc_main)

    for toss_count in range(self.total_coin_toss):
         
      ## check what numbers we are currently on
      print('The current position of quantum player is: ', self.current_position) 
      print('The current position of classical player is: ', self.current_classical_position)

      ## toss a coin
      # tail = 1 step ahead, head = 2 steps ahead 
      # use a random generator to generate 1 or 2
      toss_value = random.randint(1,2)

      print('The value of the toss is: ', toss_value)

      # position of ladders: 1 --> 5, 4 --> 13
      # position of snakes: 10 --> 3, 14 --> 0

      for num_positions in range(len(self.current_position)):
        self.current_position[num_positions] = self.current_position[num_positions] + toss_value
        circ_main = circ_main.compose(self.adder(toss_value))

      self.current_classical_position[0] = self.current_classical_position[0] + toss_value

      print('After the toss, the current position of quantum player is: ', self.current_position)
      print('After the toss, the current position of classical player is: ', self.current_classical_position)

      print('Checking for ladders and snakes....')
      
      ## apply appropriate operator based on the coin toss

      for num_positions in range(len(self.current_position)):
        if self.current_position[num_positions]==1:
          circ_main = circ_main.compose(self.ladder(1,5))
          self.current_position = self.current_position + [5]
          self.current_classical_position[0] = 5
          print('There is a ladder at position 1 and the current quantum player position is: ', self.current_position)
          print('There is a ladder at position 1 and the current classical player position is: ', self.current_classical_position) 
        elif self.current_position[num_positions]==4:
          circ_main = circ_main.compose(self.ladder(4,13))
          self.current_position = self.current_position + [13]
          self.current_classical_position[0] = 13
          print('There is a ladder at position 4 and the current quantum player position is: ', self.current_position) 
          print('There is a ladder at position 4 and the current classical player position is: ', self.current_classical_position) 
        elif self.current_position[num_positions]==10:
          circ_main = circ_main.compose(self.ladder(10,3))
          self.current_position[num_positions] = 3
          self.current_classical_position[0] = 3
          print('There is a snake at position 10 and the current quantum player position is: ', self.current_position) 
          print('There is a snake at position 10 and the current classical player position is: ', self.current_classical_position) 
        elif self.current_position[num_positions]==14:
          circ_main = circ_main.compose(self.ladder(14,0))
          # current_position = current_position.remove(14) + [0]
          self.current_position[num_positions] = 0
          self.current_classical_position[0] = 0
          print('There is a snake at position 14 and the current quantum player position is: ', self.current_position) 
          print('There is a snake at position 14 and the current classical player position is: ', self.current_classical_position) 


  def dec_to_bin(self, n):
    bnr = bin(n).replace('0b','')
    x = bnr[::-1] #this reverses an array
    while len(x) < self.Max_bits:
      x += '0'
    bnr = x[::-1] 
    return bnr 
    
  def adder(self, n):
    """
    n: number of adder operations, corresponding to the number of steps forward in the game
    """
    qr = QuantumRegister(self.Max_bits, name='q')
    adder_circ = QuantumCircuit(qr)
    for i in range(n):
      for j in reversed(range(1, self.Max_bits)):
        adder_circ.mct(list(range(j)), j)
      adder_circ.x(0)

    return adder_circ

  def ladder(self, start_pt, end_pt):
    """
      creates a ladder from start_pt to end_pt. 
      In the quantum mechanical game, ladder creates a superpostion between these
      points.
    """
    start = self.dec_to_bin(start_pt)
    end = self.dec_to_bin(end_pt)

    qr = QuantumRegister(self.Max_bits, name='q')
    anc = QuantumRegister(1, name='anc')
    circ = QuantumCircuit(qr, anc)

    for i in range(self.Max_bits):
      if start[i] == '0':
        circ.x(qr[self.Max_bits - 1 -i])
    circ.mct(list(range(self.Max_bits)), anc[0])
    for i in range(self.Max_bits):
      if start[i] == '0':
        circ.x(qr[self.Max_bits-1-i])
    circ.barrier()
    total_diff = 0
    for i in range(self.Max_bits):
      if (start[i] == '1' and end[i] == '0') or (start[i] == '0' and end[i] == '1'):
        total_diff+=1
        if total_diff == 1:
          circ.ch(anc[0],qr[self.Max_bits-1-i])
          j = i
        else:
          circ.mct([anc[0],qr[self.Max_bits-1-j]],qr[self.Max_bits-1-i])
    # reset ancilla
    circ.reset(anc)

    return circ

  def snake(self, head, tail):
    """
      creates a snake from head to tail. 
      In the quantum mechanical game, snake changes the superpostion of any state
      with head to a superposition between that state and tail.
    """
    start = self.dec_to_bin(head)
    end = self.dec_to_bin(tail)

    qr = QuantumRegister(self.Max_bits, name='q')
    anc = QuantumRegister(1, name='anc')
    circ = QuantumCircuit(qr, anc)

    # snake from head to tail
    for i in range(self.Max_bits):
      if start[i] == '0':
        circ.x(qr[self.Max_bits-1-i])
    circ.mct(list(range(self.Max_bits)), anc[0])
    for i in range(self.Max_bits):
      if start[i] == '0':
        circ.x(qr[self.Max_bits-1-i])

    circ.barrier()

    # snake from head to tail
    for i in range(self.Max_bits):
      if (start[i] == '1' and end[i] == '0') or (start[i] == '0' and end[i] == '1'):
        circ.cx(anc[0],qr[self.Max_bits-1-i])

    # reset ancilla
    circ.reset(anc)

    return circ #circ.to_instruction()



In [74]:
newgame = quantum_snake_ladder()
print(newgame.current_position)
print(newgame.current_classical_position)

The current position of quantum player is:  [0]
The current position of classical player is:  [0]
The value of the toss is:  1
After the toss, the current position of quantum player is:  [1]
After the toss, the current position of classical player is:  [1]
Checking for ladders and snakes....
There is a ladder at position 1 and the current quantum player position is:  [1, 5]
There is a ladder at position 1 and the current classical player position is:  [5]
The current position of quantum player is:  [1, 5]
The current position of classical player is:  [5]
The value of the toss is:  1
After the toss, the current position of quantum player is:  [2, 6]
After the toss, the current position of classical player is:  [6]
Checking for ladders and snakes....
The current position of quantum player is:  [2, 6]
The current position of classical player is:  [6]
The value of the toss is:  1
After the toss, the current position of quantum player is:  [3, 7]
After the toss, the current position of clas